# Libbitcoin BX: Transactions with relative locktime.
In this BX tutorial, we will send funds to a `checksequenceverify` encumbered output, and spend this after the locktime delay (which begins when the uxto is included in a block) has expired.
<img src="images/csv_overview.jpg" alt="drawing" style="" width="700px"/>

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending key pairs.


In [1]:
# my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)


### Restore your spending keys.

* `m/44'/1'/1'/0/5`

In [2]:
# hd_m_44h_1h_1h_0_5=

# privatekey_44h_1h_1h_0_5=
# publickey_44h_1h_1h_0_5=
# publickeyhash_44h_1h_1h_0_5=
# address_44h_1h_1h_0_5=

hd_m_44h_1h_1h_0_5=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 5)

privatekey_44h_1h_1h_0_5=$(bx hd-to-ec $hd_m_44h_1h_1h_0_5)
publickey_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_5=$(bx sha256 $publickey_44h_1h_1h_0_5 | bx ripemd160)
address_44h_1h_1h_0_5=$(bx hd-to-public $hd_m_44h_1h_1h_0_5 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_5
echo $publickeyhash_44h_1h_1h_0_5


mqpVyrj3iRPe7sNKht9vvnbhnDCAPdiJYv
7102c7914f7ac4e5a8055c5954b3462e0ddb419f


### Generate destination keys to send funds to.

* `m/44'/1'/2'/0/5`

In [3]:
# hd_m_44h_1h_2h_0_5=

# privatekey_44h_1h_2h_0_5=
# publickey_44h_1h_2h_0_5=
# publickeyhash_44h_1h_2h_0_5=
# address_44h_1h_2h_0_5=

hd_m_44h_1h_2h_0_5=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 2  \
| bx hd-private --index 0 \
| bx hd-private --index 5)

privatekey_44h_1h_2h_0_5=$(bx hd-to-ec $hd_m_44h_1h_2h_0_5)
publickey_44h_1h_2h_0_5=$(bx hd-to-public $hd_m_44h_1h_2h_0_5 | bx hd-to-ec)
publickeyhash_44h_1h_2h_0_5=$(bx sha256 $publickey_44h_1h_2h_0_5 | bx ripemd160)
address_44h_1h_2h_0_5=$(bx hd-to-public $hd_m_44h_1h_2h_0_5 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_2h_0_5
echo $publickeyhash_44h_1h_2h_0_5


msVCynJ5Hr5UoXE3UKMFQEhHHCajQraSK9
834cef9529c1c72fb211cf7c202d521824c9b56b


### Fetch previous UTXO's you are spending.

In [1]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"



In [2]:
# previous_txid=
# previous_output_index=
# previous_output_amount=



<hr style="border: 0.5px dashed #000;">

## 2. Spending to a CSV encumbered output<br>
<img src="images/spend_to_csv.jpg" alt="drawing" style="" width="700px"/>

### A. Build transaction with CSV encumbered output for signing

#### Determine delay in blocks
* Encode delay value in hex (little endian)
* `CHECKSEQUENCEVERIFY` verifies a 3-byte `[delay]` value on the stack against the spending transaction<br>
* For a `[delay]` in blocks:
    * Bit flag (1 << 22) is not set.
    * First 16 bits interpreted as blocks.
    * Little endian encoded.  

In [3]:
# A five block delay: 0x000005

# delay=




**Note:** Don't forget to flip byte-order (3-Bytes) above for output script below.

#### Encode CSV-encumbered P2PKH output script
* **`[delay]`**
* **`CHECKSEQUENCEVERIFY`**
* `DROP`
* `DUP`
* `HASH160`
* `[public key hash]`
* `EQUALVERIFY`
* `CHECKSIG`

In [4]:
# output_script=



#### Calculate output amount

* `400 Satoshi/sigops + 1 Satoshi/Byte` 
* Output amount(s) are previous output amount(s) minus fees

In [5]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+(6+25)) + 4 

# tx_byte_count=



In [6]:
# Output amount total.

# output_amount=



#### Encode transaction for signing
* `bx tx-encode -i [prevout txid:index:sequence] -o [output script:amount]`


In [7]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# csv_tx=




### Sign the transaction.


**Fetch previous output script**

In [8]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=



**Sign transaction with previous output script.**

In [9]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 


**Set the input script into the finalised transaction.**

In [10]:
# bx input-set "[signature] [public key point]" [transaction template]

# csv_tx=


In [11]:
# Decode.


In [12]:
# Validate.


### Validate & Broadcast the endorsed transaction
* `bx validate-tx` `[encoded transaction]`
* `bx tx-decode -f json` `[encoded transaction]` | `jq ".transaction.hash"`

In [ ]:
# Validate.


In [ ]:
# Get hash.


In [ ]:
# Send.


<hr style="border: 0.5px dashed #000;">

## 3. Spend the CSV encumbered UXTO

<img src="images/spend_from_csv.jpg" alt="drawing" style="" width="700px"/>

### A. Build p2pkh output.

#### Encode p2pkh output script

* Please send testnet coins to the following address: `n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`

In [33]:
bx address-decode -f json n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw | jq ".wrapper.payload"


"e48199d47742b245464b1366d95ef26aa4c8bb2c"


In [34]:
# output_script=



76a914e48199d47742b245464b1366d95ef26aa4c8bb2c88ac


**Encode transaction with delay & set input sequence**<br>
* Transaction sequence delay must be `>=` delay in CSV output being spent.
* Sequence delay encoding in blocks:
    * Transaction version must be >= 2.
    * Bit flag (1 << 22) must be deactivated.
    * Encode block delay in first 16 bits.

In [35]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+25) + 4 

# tx_byte_count=



192


In [36]:
# Output amount total.

# output_amount=



498958


In [38]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

# my_tx=



### Sign the transaction

**Fetch previous output script**


In [13]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=


**Sign transaction with previous output script.**

In [14]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature= 



**Set the input script into the finalised transaction.**

In [15]:
# bx input-set "[signature] [public key point]" [transaction template]

# my_tx=



In [16]:
# Validate.


In [17]:
# Decode
